# Config stuff

In [1]:
import ConnectionConfig as cc
from delta import DeltaTable
cc.setupEnvironment()

# Start local cluster

In [2]:
spark = cc.startLocalCluster("FACT_RIDES")
spark.getActiveSession()

# Create facts table: rides

## Read from sources

### Read from VeloDB database

In [13]:
#EXTRACT

cc.set_connectionProfile("VeloDB")

# Read rides table from source VeloDB database
# Only rows after 2019-01-01 will be read because older rows are corrupt
rides_source_df = spark.read \
    .format("jdbc") \
    .option("url", cc.create_jdbc()) \
    .option("driver" , cc.get_Property("driver")) \
    .option("dbtable", "(select rideid, starttime, endtime, subscriptionid, startlockid, endlockid, round(haversine_km(startpoint[0] :: numeric, startpoint[1]:: numeric, endpoint[0]:: numeric, endpoint[1]:: numeric),3) as distance_km_MV from rides where starttime > '2019-01-01') as subq") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .option("partitionColumn", "rideid") \
    .option("numPartitions", 4) \
    .option("lowerBound", 0) \
    .option("upperBound", 4140000) \
    .load()

rides_source_df.show(10)

+------+-------------------+-------------------+--------------+-----------+---------+--------------------+
|rideid|          starttime|            endtime|subscriptionid|startlockid|endlockid|      distance_km_mv|
+------+-------------------+-------------------+--------------+-----------+---------+--------------------+
|    15|2019-09-22 08:46:43|2019-09-22 09:01:36|         13296|       4849|     3188|3.443000000000000000|
|    16|2019-09-22 08:19:51|2019-09-22 08:21:55|         45924|       NULL|     NULL|0.487000000000000000|
|    17|2019-09-22 08:27:38|2019-09-22 08:30:25|         25722|       2046|     1951|0.699000000000000000|
|    18|2019-09-22 08:41:48|2019-09-22 08:46:52|         31000|       1821|     2186|1.406000000000000000|
|    19|2019-09-22 08:50:08|2019-09-22 09:09:02|         59732|       6382|     2700|4.884000000000000000|
|    20|2019-09-22 08:29:42|2019-09-22 08:31:40|          NULL|       NULL|     NULL|0.509000000000000000|
|    21|2019-09-22 08:05:17|2019-09-2

### Read from deltatables

In [14]:
#EXTRACT
# Dimension date
dim_date = spark.read.format("delta").load("spark-warehouse/dimdate")

# Dimension weather
dim_weather = spark.read.format("delta").load("spark-warehouse/dimweather")

# Dimension customer
dim_customer = spark.read.format("delta").load("spark-warehouse/dimuser")

# Dimension lock
dim_lock = spark.read.format("delta").load("spark-warehouse/dimlock")

### Read from weather data source

In [15]:
#EXTRACT
weather_responses = spark.read.format("json").option("multiLine",True).load("weather")
weather_responses.show(10)

+--------+------+---+--------------+----------+-------+--------------------+-----+------+--------------------+--------+----------+--------------------+-----------------+-------+
|    base|clouds|cod|         coord|        dt|     id|                main| name|  rain|                 sys|timezone|visibility|             weather|             wind|zipCode|
+--------+------+---+--------------+----------+-------+--------------------+-----+------+--------------------+--------+----------+--------------------+-----------------+-------+
|stations| {100}|200|{44.34, 10.99}|1583593967|3163858|{298.74, 933, 64,...|Zocca|{3.16}|{IT, 2075663, 166...|    7200|     10000|[{overcast clouds...|{349, 1.18, 0.62}|   2000|
|stations| {100}|200|{44.34, 10.99}|1583132121|3163858|{298.74, 933, 64,...|Zocca|{3.16}|{IT, 2075663, 166...|    7200|     10000|[{overcast clouds...|{349, 1.18, 0.62}|   2018|
|stations| {100}|200|{44.34, 10.99}|1583134645|3163858|{298.74, 933, 64,...|Zocca|{3.16}|{IT, 2075663, 166...|

## Create tempviews

In [16]:
# Rides source table
rides_source_df.createOrReplaceTempView("ridesSource")

# Dimension date
dim_date.createOrReplaceTempView("dimDate")

# Dimension weather
dim_weather.createOrReplaceTempView("dimWeather")

# Weather responses
weather_responses.createOrReplaceTempView("weatherResponses")

# Dimension customer
dim_customer.createOrReplaceTempView("dimCustomer")

# Dimension lock
dim_lock.createOrReplaceTempView("dimLock")

## Transform weather responses table

Add weather_ID based on the weather type (condition_id)

Condition id:
- < 800: All codes with a number smaller than 800 means rain in some form. (=onaangenaam code 2)
- = 800: This code means clear sky and sunshine (=aangenaam code 1 if temperature is higher than 15 degrees Celsius)
- \> 800: All other weather conditions (Neutraal code 3)

In [17]:
#TRANSFORM
short_weather_responses = spark.sql("select zipCode as zip_code, dt as timestamp, weather.id[0] as condition_id, main.temp as temperature, \
                                    case \
                                        when condition_id < 800 then 2 \
                                        when condition_id = 800 and main.temp > (273 + 15) then 1 \
                                        when condition_id = 800 and main.temp < (273 + 15) then 3 \
                                        when condition_id > 800 then 3 \
                                    else 4 \
                                    end as weather_ID \
                                    from weatherResponses")
short_weather_responses.show(10)

+--------+----------+------------+-----------+----------+
|zip_code| timestamp|condition_id|temperature|weather_ID|
+--------+----------+------------+-----------+----------+
|    2000|1583593967|         804|     298.48|         3|
|    2018|1583132121|         804|     298.48|         3|
|    2020|1583134645|         804|     298.48|         3|
|    2030|1585973548|         804|     298.48|         3|
|    2050|1569163778|         804|     298.48|         3|
|    2060|1569968446|         804|     298.48|         3|
|    2100|1575264035|         804|     298.48|         3|
|    2140|1619936979|         804|     298.48|         3|
|    2170|1656838375|         804|     298.48|         3|
|    2600|1659778730|         804|     298.48|         3|
+--------+----------+------------+-----------+----------+
only showing top 10 rows



In [18]:
short_weather_responses.createOrReplaceTempView("shortWeatherResponses")

## Build facts table

In [19]:
#TRANSFORM
rides_fact_df = spark.sql("select src.rideid as ride_ID, dd.date_SK, \
                          coalesce(dw.weather_SK, 3) as weather_SK, \
                          dls.lock_SK as start_lock_SK, dle.lock_SK as end_lock_SK, \
                          dc.user_sk,\
                          1 as count_MV, \
                          (unix_timestamp(endtime) - unix_timestamp(starttime)) as rideDuration_MV, \
                          src.distance_km_MV as distance_km_MV,\
                          md5(concat(src.rideid, dd.date_SK, coalesce(start_lock_SK,0), coalesce(end_lock_SK,0), dc.user_sk,coalesce(dw.weather_SK, 3), 1, rideDuration_MV, distance_km_MV)) as md5 \
                          from ridesSource as src \
                          left outer join dimDate as dd \
                          on cast(src.starttime as DATE) = cast(dd.CalendarDate as DATE) \
                          left outer join dimLock dls on src.startlockid = dls.lockid \
                          left outer join dimLock dle on src.endlockid = dle.lockid \
                          left outer join shortWeatherResponses wr on dls.zipcode = string(wr.zip_code) \
                          and date_format(src.starttime, 'yyyy-MM-dd-HH') = date_format(from_unixtime(wr.timestamp),'yyyy-MM-dd-HH') \
                          left outer join dimWeather dw on dw.weather_id = wr.weather_ID \
                          left outer join dimCustomer dc on src.subscriptionid = dc.subscriptionid and src.starttime > dc.scd_start and src.endtime <= dc.scd_end\
                          where src.subscriptionid is not null and dc.userid is not null")

rides_fact_df.show(10)

+-------+-------+----------+--------------------+--------------------+--------------------+--------+---------------+--------------------+--------------------+
|ride_ID|date_SK|weather_SK|       start_lock_SK|         end_lock_SK|             user_sk|count_MV|rideDuration_MV|      distance_km_MV|                 md5|
+-------+-------+----------+--------------------+--------------------+--------------------+--------+---------------+--------------------+--------------------+
|     15|     21|         3|85d6fe29-6d66-45f...|3951a9e6-0311-42b...|1608f60a-a80e-41f...|       1|            893|3.443000000000000000|2f748218092a94dc5...|
|     17|     21|         3|32cc58c0-f13c-4a3...|b8641935-ebfd-483...|7c5a0ab8-8828-4ed...|       1|            167|0.699000000000000000|7c0e99eb9fa893f78...|
|     19|     21|         3|94e738fe-fcae-421...|3ad8e437-5361-465...|d54eeaf2-a068-475...|       1|           1134|4.884000000000000000|17ef7dde5a0df2b81...|
|     22|     21|         3|246d1092-6f3b-490.

### Create temview from facts table

In [20]:
rides_fact_df.createOrReplaceTempView("factRides_new")

## Write facts table to delta table: Initial load

In [11]:
# LOAD
rides_fact_df.write.format("delta").mode("overwrite").saveAsTable("factRides")

## Incremental load

In [21]:
#LOAD
dt_factRides = DeltaTable.forPath(spark,".\spark-warehouse\\factrides")
dt_factRides.toDF().createOrReplaceTempView("factRides_current")

result = spark.sql("merge into factRides_current as target \
                   using factRides_new as source on target.ride_ID = source.ride_ID \
                   when matched and (target.md5 <> source.md5) then update set * \
                   when not matched then insert *")

result.show()

+-----------------+----------------+----------------+-----------------+
|num_affected_rows|num_updated_rows|num_deleted_rows|num_inserted_rows|
+-----------------+----------------+----------------+-----------------+
|                0|               0|               0|                0|
+-----------------+----------------+----------------+-----------------+



In [28]:
spark.stop()